In [4]:
import sys, os
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
from tensorboard.plugins import projector

def visualize(model, TENSORBOARD_DIR):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), dimention))

    with open(os.path.join(TENSORBOARD_DIR,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    embeddings = tf.Variable(placeholder, name='embeddings')
    CHECKPOINT_FILE = TENSORBOARD_DIR + 'w2x_metadata.ckpt'
    print(CHECKPOINT_FILE)
    ckpt = tf.train.Checkpoint(embeddings=embeddings)
    ckpt.save(CHECKPOINT_FILE)

    reader = tf.train.load_checkpoint(TENSORBOARD_DIR)
    map = reader.get_variable_to_shape_map()
    key_to_use = ""
    for key in map:
        if "embeddings" in key:
            key_to_use = key

    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = key_to_use
    embedding.metadata_path = meta_file

    projector.visualize_embeddings(TENSORBOARD_DIR, config)
    print('Run `tensorboard --logdir={0} --host 192.168.1.41 --port 6006` to run visualize result on tensorboard'.format(TENSORBOARD_DIR))
    
# if __name__ == "__main__":
model = Word2Vec.load("model/word2vec_model_300")
visualize(model,"visualize/")

/usr/local/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


visualize/w2x_metadata.ckpt
Run `tensorboard --logdir=visualize/` to run visualize result on tensorboard
